In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import torch

from experiments import *
import autoencoders as ae
import wandb

## Full-contrastive experiments

In [ ]:
wandb.init(project="contrastive test (full)")

In [ ]:
suite = ae.TrajectoryContrastiveSuite(Pendulum, contrastive_loss=ae.lifted_structured_loss(5, 1, 1), batch_size=128, epochs=500, do_animate=True)
model = suite.train(40, 200)

In [ ]:
loss, mse = suite.test_num(model)
print(loss, mse)

In [ ]:
x, x_prime, dyn, cons = suite.try_transform_num(model, 2)
print(cons.min(), cons.max())
plt.scatter(x[:, 0], x[:, 1], c='black', s=0.01)
plt.scatter(x_prime[:, 0], x_prime[:, 1], c=dyn, s=1)

In [ ]:
x, x_prime, dyn, cons = suite.try_transform_num(model, 2)
print(cons.min(), cons.max())
plt.scatter(x[:, 0], x[:, 1], c='black', s=0.01)
plt.scatter(x_prime[:, 0], x_prime[:, 1], c=dyn, s=1)

## UMAP & Wasserstein experiments
Coupled oscillator for now

In [ ]:
exp = CoupledOscillator
target_embedding, dist_matrix = exp.calc_umap_wd(200, 200)

### UMAP experiments

In [ ]:
wandb.init(project="contrastive test (umap)")

In [ ]:
target_embedding_tensor = torch.tensor(target_embedding).float()

def umap_mse(embedding, labels, lambda_=0.1):
    global target_embedding_tensor
    return torch.mean((embedding - target_embedding_tensor[labels.long(),])**2) * lambda_

In [ ]:
suite = ae.TrajectoryContrastiveSuite(exp, contrastive_loss=umap_mse, batch_size=128, epochs=2000, do_animate=True, optim_config={"step_size": 500, "gamma": 0.5})
model = suite.train(80, 200)

In [ ]:
x = exp.contrastive_data(20, 20)
data = x[:, 1:]
labels = x[:, 0]
emb = model.encoder(torch.tensor(data).float())[:, 2:]

In [ ]:
loss, mse = suite.test_num(model)
print(loss, mse)

In [ ]:
x = exp.data[0]
x_prime, dyn, cons = suite.try_transform(model, x)
print(cons[:, 0].min(), cons[:, 0].max())
plt.scatter(x[:, 0], x[:, 1], c='black', s=0.01)
plt.scatter(x_prime[:, 0], x_prime[:, 1], c=dyn[:, 0], s=1)
dyn1 = dyn

In [ ]:
x, x_prime, dyn, cons = suite.try_transform_num(model, 3)
print(cons[:, 0].min(), cons[:, 0].max())
plt.scatter(x[:, 0], x[:, 1], c='black', s=0.01)
plt.scatter(x_prime[:, 0], x_prime[:, 1], c=dyn[:, 0], s=1)
dyn3 = dyn

In [ ]:
plt.scatter(dyn1[:, 0], dyn1[:, 1], s=1, label="traj 1")
plt.scatter(dyn2[:, 0], dyn2[:, 1], s=1, label="traj 2")
plt.scatter(dyn3[:, 0], dyn3[:, 1], s=1, label="traj 3")
plt.legend()
plt.xlabel("dyn0")
plt.ylabel("dyn1")
plt.title("Inner dynamic embedding")

### Wasserstein experiments

In [ ]:
dist_matrix_tensor = torch.tensor(dist_matrix).float()

def wasserstein_mse(embedding, labels):
    global dist_matrix_tensor
    center_loss = ae.center_loss(embedding, labels)
    dist_loss = ae.diff_between_distances(embedding, labels, dist_matrix_tensor)
    return (center_loss + dist_loss) * 0.01

In [ ]:
wandb.init(project="contrastive test (wasserstein)")

In [ ]:
suite = ae.TrajectoryContrastiveSuite(exp, batch_size=256, contrastive_loss=wasserstein_mse, epochs=1000, do_animate=True, init_lr=0.03)
# data = exp.contrastive_data(1)
model = suite.train(80, 200)
# model = suite.train_traj_data(data)

In [ ]:
loss, mse = suite.test_num(model)
print(loss, mse)

In [ ]:
x = exp.data[0]
x_prime, dyn, cons = suite.try_transform(model, x)
print(cons[:, 0].min(), cons[:, 0].max())
plt.scatter(x[:, 0], x[:, 1], c='black', s=0.01)
plt.scatter(x_prime[:, 0], x_prime[:, 1], c=dyn[:, 0], s=1)

In [ ]:
import creating.coupled_oscillator as co

In [ ]:
x = exp.contrastive_data()[:, 1:]
x_prime, dyn, cons = suite.try_transform(model, x)
print(cons[:, 0].min(), cons[:, 0].max())
cons0, cons1 = cons[:, 0], cons[:, 1]
y_true = [co.energy2(e) for e in x]
plt.scatter(cons0, cons1, c=y_true, s=0.01)
plt.scatter(cons0[:1000], cons1[:1000], c=y_true[:1000], s=10)
plt.xlabel("cons0")
plt.ylabel("cons1")
plt.title("Coupled oscillator projection (colored by energy2)")

In [ ]:
# x = exp.contrastive_data()[:, 1:]
# x_prime, dyn, cons = suite.try_transform(model, x)
# print(cons[:, 0].min(), cons[:, 0].max())
# cons0, cons1 = cons[:, 0], cons[:, 1]
y_true = [co.energy2(e) for e in x]
# plt.scatter(cons0, cons1, c=y_true, s=0.01)
plt.scatter(target_embedding[:200, 0], target_embedding[:200, 1], c=y_true[::1000][:200], s=10)
plt.xlabel("cons0")
plt.ylabel("cons1")
plt.title("Coupled oscillator UMAP (colored by energy2)")